In [ ]:
import tensorflow as tf 
from tensorflow.keras import layers, models, metrics

import numpy as np 
import scipy as sp 
import pandas as pd
from  statsmodels.tsa.api import acf
import matplotlib.pyplot as plt

In [ ]:
%reload_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, "../")

import midiio
import frequency_analysis as frqa
from midi.midi_representation import *

In [ ]:
MIDI_START = 20
MIDI_END = 127

FREQUENCY_COUNT = 257

In [ ]:
# Let us define our input and output data
input_columns = ["f" + str(i) for i in range(0, FREQUENCY_COUNT)]
label_columns = [i for i in range(MIDI_START, MIDI_END+1)]

In [ ]:
def get_data(wav_path, csv_path):
    # Open the midifile to see what is inside. Also open the accopmanying labels.
    data, sample_rate = midiio.read_file(wav_path)

    music_dataframe = pd.read_csv(csv_path)

    music_dataframe
    WINDOW_SAMPLES = sample_rate * 0.05

    # For the model, construct the training data as follows:
    music_dataframe_copy = music_dataframe.copy()
    training_data = pd.DataFrame(columns = ['start_time'] + label_columns + input_columns)

    # Create a linear series where points are WINDOW_SAMPLES apart from each other. 
    serialized_data = []
    xs = np.linspace(0, len(data), int (len(data) / WINDOW_SAMPLES), endpoint=False)

    # Iterate over the music dataframe. Construct a one hot encoded vector for this particular time based on the note value
    # At the given time.
    i = 0

    for x in xs:
        notes_on = music_dataframe_copy.query("start_time <= " + str(int(x))).query("end_time >= " + str(int(x)))
        music_dataframe_copy.drop(notes_on.index, axis='index', inplace=True)
        
        note_vec = [0 for i in range(MIDI_START, MIDI_END + 1)]
        if len(notes_on) != 0:
            for n in notes_on['note']:
                note_vec[n - MIDI_START] = 1
        
        
        f, power = frqa.get_frequencies(data[int(x) : int(x + WINDOW_SAMPLES)], sample_rate)
        training_data.loc[len(training_data.index)] = [x, *note_vec, *np.abs(power)]

    return training_data

In [ ]:
training_data = get_data("../data/musicnet/1727.wav", "../data/musicnet/1727.csv").append(
                get_data("../data/musicnet/1759.wav", "../data/musicnet/1759.csv"))
training_data